# Curso NLP + Transformers

<img src="https://yaelmanuel.com/wp-content/uploads/2021/12/platzi-banner-logo-matematicas.png" width="500px">

---

# Análisis de Reseñas de Amazon 📦🔍

## 1) Carga y Exploración del Dataset 🤓

**Objetivo: Exploración del Dataset.**

In [1]:
import pandas as pd

Descomprimir el archivo .rar

In [2]:
!unrar x "/content/reviews_clean_advanced.rar"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/reviews_clean_advanced.rar

Extracting  reviews_clean_advanced.csv                                    70%100%  OK 
All OK


Carga del dataset

In [3]:
csv_path = "/content/reviews_clean_advanced.csv"
filter_data = pd.read_csv(csv_path)

Visualizamos las primeras filas del dataset

In [4]:
filter_data.head(3)

,clean_review_lemmatization,stars
0,bueno kar pantalla menos mes recibir respues...,1
1,horrible comprar saber ingls informtico despu ...,1
2,obligar comprar dos unidad llegar solo forma r...,1


Evaluar los valores nulos

In [5]:
filter_data.isnull().sum()

,0
clean_review_lemmatization,1
stars,0


In [6]:
filter_data = filter_data.dropna()

## 2) Representación Vectorial: Bag-of-Words y TF-IDF 📁

**Objetivo: Convertir las reseñas en representaciones numéricas mediante Bag-of-Words y TF-IDF para su posterior análisis.**

**Bag-of-Words (BoW):** Es una técnica que convierte texto en una representación numérica al contar la frecuencia de cada palabra en un documento, ignorando el orden y la gramática.

Cada documento se representa como un vector donde cada dimensión corresponde a una palabra del vocabulario y el valor es la frecuencia de esa palabra en el documento.

**TF-IDF (Term Frequency-Inverse Document Frequency):** Esta técnica mejora la representación BoW al ponderar la frecuencia de las palabras por su importancia en el corpus.

Calcula la frecuencia de una palabra en un documento (TF) y la multiplica por la inversa de la frecuencia de documentos que contienen esa palabra (IDF), reduciendo la influencia de palabras comunes y destacando términos más informativos.


In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Crear el corpus a partir de las reseñas limpias

In [8]:
corpus = filter_data["clean_review_lemmatization"].tolist()

### 📍 Representación Bag-of-Words con CountVectorizer

In [ ]:
cv = CountVectorizer()
bow_matrix = cv.fit_transform(corpus)
print("Dimensiones de la matriz Bag-of-Words:", bow_matrix.shape)
print("Ejemplo de términos (BoW):", cv.get_feature_names_out()[:10])

Dimensiones de la matriz Bag-of-Words: (209999, 61223)
Ejemplo de términos (BoW): ['aa' 'aaa' 'aaaa' 'aaaaa' 'aaaaaa' 'aaao' 'aac' 'aad' 'aada' 'aadar']


### 📍 Representación TF-IDF con TfidfVectorizer

Examinar las palabras con mayor peso TF-IDF para identificar términos relevantes en el texto.

In [9]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(corpus)
print("Dimensiones de la matriz TF-IDF:", tfidf_matrix.shape)
print("Ejemplo de términos (TF-IDF):", tfidf.get_feature_names_out()[:10])

Dimensiones de la matriz TF-IDF: (209999, 61223)
Ejemplo de términos (TF-IDF): ['aa' 'aaa' 'aaaa' 'aaaaa' 'aaaaaa' 'aaao' 'aac' 'aad' 'aada' 'aadar']


## 3) Extracción de Términos Clave y Modelado de Temas 🔍

**Objetivo: Utilizar LDA para extraer temas y términos clave de las reseñas.**

**Modelado de temas con LDA (Latent Dirichlet Allocation)**: LDA es una técnica de modelado generativo que asume que cada documento es una mezcla de temas y que cada tema es una mezcla de palabras. Ayuda a descubrir temas ocultos en una colección de documentos.

**Extracción de palabras clave:** Métodos como la frecuencia de términos, TF-IDF y algoritmos como RAKE (Rapid Automatic Keyword Extraction) se utilizan para identificar palabras o frases que capturan la esencia de un documento.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

Aplicar LDA sobre la matriz Bag-of-Words para extraer 5 temas

In [ ]:
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(bow_matrix)

LatentDirichletAllocation(n_components=5, random_state=42)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Tema %d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
display_topics(lda, cv.get_feature_names_out(), 10)

Tema 0:
llegar producto él venir devolver dar poder amazon caja si
Tema 1:
funcionar bien él luz dar dejar cable batera ms ir
Tema 2:
bien él si hacer poder quedar ir pantalla poner ms
Tema 3:
calidad buen precio bien producto ms cumplir bonito perfecto color
Tema 4:
bien ms él pequeo perfecto grande comprar gustar tamao si


## 4) Clasificación Tradicional para Análisis de Sentimientos y Categorías 👍 👎

**Objetivo: Entrenar y evaluar un clasificador (Naive Bayes) para determinar el sentimiento (positivo/negativo) de las reseñas.**

**Naive Bayes:** Es un clasificador probabilístico basado en el teorema de Bayes, que asume la independencia entre las características.

**Calcula la probabilidad de que un documento pertenezca a una clase (por ejemplo, positivo o negativo) asumiendo que las características (palabras) son independientes entre sí.**

Esta suposición "ingenua" simplifica mucho los cálculos y permite entrenar y predecir rápidamente, algo muy útil en entornos donde el tiempo y los recursos pueden ser limitados.


Definición: Se considera reseña positiva cuando la puntuación ("stars") es mayor que 3; negativa en caso contrario.

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

Crear la variable binaria de sentimiento: 1 (positivo) si stars > 3, 0 (negativo) de lo contrario

In [10]:
filter_data["sentiment_bin"] = filter_data["stars"].apply(lambda x: 1 if x > 3 else 0)

Usar la representación TF-IDF para el modelo

In [12]:
X = tfidf_matrix
y = filter_data["sentiment_bin"]

Dividir el dataset en entrenamiento y prueba

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Entrenar el clasificador Naive Bayes

In [14]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

MultinomialNB()

Evaluar el modelo

In [15]:
y_pred = nb_classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Accuracy: 0.7911190476190476
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.78      0.92      0.84     25204
           1       0.83      0.60      0.70     16796

    accuracy                           0.79     42000
   macro avg       0.80      0.76      0.77     42000
weighted avg       0.80      0.79      0.78     42000



**Guardar el modelo**

In [16]:
import joblib

model_path = "/content/nb_classifier_model.pkl"
joblib.dump(nb_classifier, model_path)

['/content/nb_classifier_model.pkl']

**Cargar el modelo**

In [17]:
mi_modelo = joblib.load(model_path)


**Probar con nueva data**

In [18]:
import re
import string


def clean(text):
    # Convertir a minúsculas
    text = str(text).lower()

    # Eliminar textos entre corchetes (ej.: etiquetas)
    text = re.sub(r'\[.*?\]', '', text)

    # Eliminar URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Eliminar etiquetas HTML
    text = re.sub(r'<.*?>+', '', text)

    # Eliminar signos de puntuación
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

    # Eliminar saltos de línea
    text = re.sub(r'\n', ' ', text)

    # Eliminar palabras que contienen números
    text = re.sub(r'\w*\d\w*', '', text)

    # Eliminar emojis y caracteres especiales (no ASCII)
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Eliminar espacios extras al inicio y final
    text = text.strip()

    return text

In [19]:
!python3 -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 116.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [20]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopword_es = set(stopwords.words('spanish'))

import spacy
nlp_es = spacy.load('es_core_news_sm')


def clean_with_stopwords_and_lemmatization(text):
    # Procesar el texto usando spaCy
    doc = nlp_es(text)
    # Eliminar stopwords y aplicar lematización
    lemmatized = [token.lemma_ for token in doc if token.text.lower() not in stopword_es]
    # Unir los tokens lematizados y eliminar espacios extra
    return " ".join(lemmatized).strip()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
# Ejemplo de nueva reseña
new_review = "Este producto es excelente y superó mis expectativas."

# Preprocesar la reseña usando la función clean + lematización
new_review_clean = clean(new_review)
new_review_clean = clean_with_stopwords_and_lemmatization(new_review_clean)

# Transformar el nuevo texto en el mismo espacio vectorial con el vectorizador TF-IDF ya ajustado
new_vector = tfidf.transform([new_review_clean])

# Realizar la predicción con el modelo cargado
prediction = mi_modelo.predict(new_vector)

# Mostrar la predicción (por ejemplo, 1 para positivo, 0 para negativo)
print("Predicción de sentimiento:", prediction[0])

Predicción de sentimiento: 1


In [22]:
# Ejemplo de nueva reseña
new_review = "Lo odio ;)!"

# Preprocesar la reseña usando la función clean + lematización
new_review_clean = clean(new_review)
new_review_clean = clean_with_stopwords_and_lemmatization(new_review_clean)

# Transformar el nuevo texto en el mismo espacio vectorial con el vectorizador TF-IDF ya ajustado
new_vector = tfidf.transform([new_review_clean])

# Realizar la predicción con el modelo cargado
prediction = mi_modelo.predict(new_vector)

# Mostrar la predicción (por ejemplo, 1 para positivo, 0 para negativo)
print("Predicción de sentimiento:", prediction[0])

Predicción de sentimiento: 0


In [23]:
# Ejemplo de nueva reseña
new_review = "me encanto?"

# Preprocesar la reseña usando la función clean + lematización
new_review_clean = clean(new_review)
new_review_clean = clean_with_stopwords_and_lemmatization(new_review_clean)

# Transformar el nuevo texto en el mismo espacio vectorial con el vectorizador TF-IDF ya ajustado
new_vector = tfidf.transform([new_review_clean])

# Realizar la predicción con el modelo cargado
prediction = mi_modelo.predict(new_vector)

# Mostrar la predicción (por ejemplo, 1 para positivo, 0 para negativo)
print("Predicción de sentimiento:", prediction[0])

Predicción de sentimiento: 1
